# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import joblib
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, AciWebservice
from azureml.core.model import Model
import requests
import json

#from azureml.core import ScriptRunConfig

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'udacity-project-hd'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code A35FP6GJ2 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-138423
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-138423


In [4]:
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
### YOUR CODE HERE ###

# Choose a name for your CPU cluster
compute_cluster_name = "computecluster"

# Verify that the cluster does not exist already

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor = 0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
        "--C": uniform(0.1, 1.0),
        "--max_iter": choice(50, 100, 200, 500, 1000, 1500)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

script_folder= './training'
os.makedirs(script_folder, exist_ok=True)

import shutil
shutil.copy('./train.py', script_folder)

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=script_folder, #'./'
              compute_target=compute_cluster,
              script_params={'--C': 1.0, '--max_iter': 100},
              pip_packages=['sklearn'],
              entry_script='train.py') #A string representing the relative path to the file used to start training.

#sklearn_env = Environment.get(workspace=ws, name="AzureML-Tutorial")

#src = ScriptRunConfig(source_directory='./',
#                      script='train.py',
#                      compute_target = compute_target,
#                      environment=sklearn_env)


hyperdrive_run_config = HyperDriveConfig(estimator=estimator, # run_config=src
                                     hyperparameter_sampling=param_sampling,
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy', 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4) #similar to the number of nodes

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [7]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config, show_output=True) 
#show_output=True <- vielleicht noch raushauen
#the show_output shows the results for the OPTIONAL (next)

The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--C', '--max_iter'] is the list of overridden parameter(s).


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_3825af57-a99f-4833-b18f-1f6a2784775e
Web View: https://ml.azure.com/experiments/udacity-project-hd/runs/HD_3825af57-a99f-4833-b18f-1f6a2784775e?wsid=/subscriptions/9e65f93e-bdd8-437b-b1e8-0647cd6098f7/resourcegroups/aml-quickstarts-138423/workspaces/quick-starts-ws-138423

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-10T18:02:23.329209][API][INFO]Experiment created<END>\n"<START>[2021-02-10T18:02:25.3859430Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-02-10T18:02:26.482664][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-02-10T18:02:26.652480][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_3825af57-a99f-4833-b18f-1f6a2784775e
Web View: https://ml.azure.com/experiments/udacity-project-hd/runs/HD_3825af57-a99f-4833-b18f-1f6a2784775e?wsid=/subscriptions

{'runId': 'HD_3825af57-a99f-4833-b18f-1f6a2784775e',
 'target': 'computecluster',
 'status': 'Canceled',
 'startTimeUtc': '2021-02-10T18:02:23.07071Z',
 'endTimeUtc': '2021-02-10T18:15:25.718933Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name": "

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [16]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()

In [15]:
# -> display all the properties of best_run model
best_run.get_details()

AttributeError: 'NoneType' object has no attribute 'get_details'

In [18]:
print(best_run.get_details()['runDefinition']['arguments'])

AttributeError: 'NoneType' object has no attribute 'get_details'

In [ ]:
metrics=best_run.get_metrics()

In [ ]:
print(metrics)

In [ ]:
print('\n Accuracy: ', metrics['Accuracy'])

In [17]:
model = best_run.register_model(model_name='SKlearn_hyperdrive', model_path='.') #'outputs/model.joblib'
print(model)

AttributeError: 'NoneType' object has no attribute 'register_model'

In [ ]:
#TODO: Save the best model
best_run.download_file(target_dir="/outputs/model.joblib", "best_model.joblib", exist_ok=True)

In [ ]:
# Nafolgendes kann evtl. entfallen.

In [ ]:
from train import clean_data
from azureml.data.dataset_factory import TabularDatasetFactory

factory = TabularDatasetFactory()
test_data_path = "https://raw.githubusercontent.com/krishula/AzureMLCapstone/main/heart.csv"
test_ds = factory.from_delimited_files(test_data_path)
X_test, y_test = clean_data(test_ds)

logit_model = joblib.load('./outputs/model.joblib')

print(logit_model.score(X_test, y_test))

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service